# Combine the performance data from individual models

Loop over models and extract focal performance to a single table.

In [ ]:
import joblib
import pandas as pd
import seaborn as sns

Loop and get the data.

In [ ]:
perf = []

for reduction in ["pca", "fa", "umap_dim20_nb5_euclidean"]:
    for model_type in ["lr", "rf"]:
        for cluster in [1, 3, 4, 5, 6, 7, 8]:
            with open(
                f"/data/uscuni-restricted/06_models/{reduction}/label_{cluster}/{model_type}/model.joblib",
                "rb",
            ) as f:
                model = joblib.load(f)
                perf.append(
                    pd.Series(
                        {
                            "reduction": reduction,
                            "model": model_type,
                            "cluster": cluster,
                            "accuracy": model.score_,
                            "balanced_accuracy": model.balanced_accuracy_,
                            "precision": model.precision_,
                            "recall": model.recall_,
                            "f1_macro": model.f1_macro_,
                            "f1_micro_": model.f1_micro_,
                            "f1_weighted": model.f1_weighted_,
                        }
                    )
                )

Combine.

In [ ]:
performance = pd.DataFrame(perf)

In [ ]:
performance.to_parquet("/data/uscuni-restricted/06_models/model_performance.parquet")

In [ ]:
ax = sns.scatterplot(
    performance, x="cluster", y="balanced_accuracy", hue="model", style="reduction"
)
sns.despine()
sns.move_legend(ax, loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

In [ ]:
ax = sns.scatterplot(
    performance, x="cluster", y="f1_macro", hue="model", style="reduction"
)
sns.despine()
sns.move_legend(ax, loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

In [ ]:
ax = sns.scatterplot(
    performance, x="cluster", y="accuracy", hue="model", style="reduction"
)
sns.despine()
sns.move_legend(ax, loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

In [ ]:
ax = sns.scatterplot(
    performance, x="cluster", y="precision", hue="model", style="reduction"
)
sns.despine()
sns.move_legend(ax, loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

In [ ]:
ax = sns.scatterplot(
    performance, x="cluster", y="recall", hue="model", style="reduction"
)
sns.despine()
sns.move_legend(ax, loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

In [ ]:
old_perf = pd.read_parquet(
    "/data/uscuni-restricted/06_models/model_performance_old.parquet"
)

In [ ]:
(
    performance.replace(
        dict(umap_dim20_nb5_euclidean="umap_dim5_nb20_euclidean")
    ).set_index(["reduction", "model", "cluster"])
    - old_perf.set_index(["reduction", "model", "cluster"])
).style.background_gradient(cmap="RdBu", vmin=-1, vmax=1)

In [ ]:
print(
    performance.set_index(["reduction", "model", "cluster"])["f1_macro"]
    .unstack()
    .round(3)
    .to_markdown()
)